In [1]:
import pandas as pd
import selenium
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException

### Q1.Write a python program which searches all the product under a particular product vertical from www.amazon.in The product vertical to be searched will be taken as input from user. For e.g. If user input is 'guitar'. Then search for guitars.

In [2]:
# opening driver 
driver=webdriver.Chrome(r'C:\Users\home\Downloads\chromedriver_win32\chromedriver.exe')
# get url
driver.get('https://www.amazon.in/')


#locating the search bar
# Locating searc_bar by id
search_bar = driver.find_element_by_id("twotabsearchtextbox")
# clearing search_bar
search_bar.clear()  
# sending user input to search bar
search_bar.send_keys("home theater") 
# Locating search_button by xpath
search_button = driver.find_element_by_xpath('//span[@id="nav-search-submit-text"]')       
search_button.click() 

### Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv.In case if any product vertical has less than 3 pages in seach results then scrape all the products available under that product vertical.Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details", and "Product URL". In case, if any of the details are missing for any of the product then replace it by "-"

In [11]:
product_url=[]
brand_name=[]
product_name=[]
Rating=[]
no_of_rating=[]
price=[]
return_replace=[]
expected_del=[]
availability=[]

time.sleep(3)
#scrapping the required details
start=0
end=4
for page in range(start,end):#for loop for scrapping 4 page
    p_url=driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]') #xpath of product url
    
    for i in p_url:
        product_url.append(i.get_attribute('href')) # appending url in product url
        
    nxt_button=driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none a-padding-base']/div/ul/li[3]/a")#scraping the list of buttons from the page
    try:
        driver.get(nxt_button[1].get_attribute('href'))#getting the link from the list for next page
    except:
        driver.get(nxt_button[0].get_attribute('href'))
for u in product_url:
    driver.get(u)
    try:
        brand=driver.find_element_by_xpath('//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span') #xpath to extract brand name
        brand_name.append(brand.text)  # appending extracted brand names in list
    except  NoSuchElementException:
        brand_name.append('-')  # if not found any brand name it replace with "-"
        
    try:
        product=driver.find_element_by_xpath('//*[@id="productOverview_feature_div"]/div/table/tbody/tr[3]/td[2]/span') #xpath to extact product name
        product_name.append(product.text) # appending extracted product name in list
    except NoSuchElementException:
        product_name.append('-') 
    try:
        rating=driver.find_element_by_xpath('//span[@class="reviewCountTextLinkedHistogram noUnderline"]')  #xpath to extract rating
        Rating.append(rating.get_attribute('title')) # appending extracted rating in list
    except NoSuchElementException:
        Rating.append('-')
    try:
        ratings=driver.find_element_by_id('acrCustomerReviewText') #xpath to extract to find no. of rating
        no_of_rating.append(ratings.text) # appending text of no. of rating in list
    except NoSuchElementException:
        no_of_rating.append('-')
    try:
        pri=driver.find_element_by_id('priceblock_ourprice')  #xpath to extract to find price
        price.append(pri.text) # appending extracted price in list
    except NoSuchElementException:
        price.append('-')
    try:
        rr=driver.find_element_by_xpath('//a[@class="a-size-small a-link-normal a-text-normal"]') # xpath to find return and replacement
        return_replace.append(rr.text) # appending extracted to list
    except NoSuchElementException:
        return_replace.append('-')
    try:
        exp_d=driver.find_element_by_xpath('//div[@class="a-section a-spacing-mini"]/b') # xpath fo find expected delivery
        expected_del.append(exp_d.text)  # appending extracted expected delivery time in list
    except NoSuchElementException:
        expected_del.append('-')
    try:
        avail=driver.find_element_by_id('availability') # xpath to check availablitiy 
        availability.append(avail.text)  # appending extracted availability in list
    except NoSuchElementException:
        availability.append('-')
        

In [13]:
df=pd.DataFrame({"Product_url":product_url,"Brand_Name":brand_name,"Product_Name":product_name,"Rating":Rating,
                "No_of_rating":no_of_rating,"Price":price,"Return_Replace":return_replace,
                "expected_delivery":expected_del,"Availability":availability})
df

,Product_url,Brand_Name,Product_Name,Rating,No_of_rating,Price,Return_Replace,expected_delivery,Availability
0,https://www.amazon.in/gp/slredirect/picassoRed...,ZEBRONICS,Wireless,3.7 out of 5 stars,55 ratings,-,7 Days Replacement,"Saturday, June 5",In stock.
1,https://www.amazon.in/gp/slredirect/picassoRed...,Artis,USB,4.3 out of 5 stars,371 ratings,₹ 629.00,7 Days Replacement,"Wednesday, June 2",In stock.
2,https://www.amazon.in/Krisons-Nexon-Bluetooth-...,-,-,3.5 out of 5 stars,520 ratings,"₹ 1,690.00",7 Days Replacement,"Saturday, June 5",In stock.
3,https://www.amazon.in/Zebronics-Subwoofer-Wire...,ZEBRONICS,AUX,2.9 out of 5 stars,9 ratings,"₹ 2,999.00",7 Days Replacement,"Saturday, June 5",In stock.
4,https://www.amazon.in/A140X-Channel-Multimedia...,F&D,"Bluetooth, USB",4.1 out of 5 stars,"2,177 ratings","₹ 2,999.00",7 Days Replacement,"Wednesday, June 2",In stock.
...,...,...,...,...,...,...,...,...,...
83,https://www.amazon.in/Tronica-BT2050-Theater-B...,-,-,4.1 out of 5 stars,49 ratings,-,7 Days Replacement,-,
84,https://www.amazon.in/Generic-Wireless-Bluetoo...,Generic,Sliver,-,-,₹ 399.00,7 Days Replacement,"Sunday, June 6",Only 1 left in stock.
85,https://www.amazon.in/IKALL-IK-666-Multimedia-...,IKALL,USB,3.9 out of 5 stars,3 ratings,"₹ 2,089.00",7 Days Replacement,"Sunday, June 6",In stock.
86,https://www.amazon.in/gp/slredirect/picassoRed...,RPMSD,"Radio, SD Card",-,-,₹ 498.00,7 Days Replacement,"Saturday, June 5",In stock.


In [14]:
driver.close()

### Q3. Write a python program to access the search bar and search button on images google.com and scrape 100 images each for keywords 'fruits','cars'and 'Machine Learning'.

In [29]:
# opening driver 
driver=webdriver.Chrome(r'C:\Users\home\Downloads\chromedriver_win32\chromedriver.exe')

In [33]:
#defining lists
Fruits_images=[]
Car_images=[]
Machine_Learning=[]

#url
url='https://www.google.co.in/imghp?hl=en&authuser=0&ogbl'

# function to sending keys to search bar
def sending(key):
    driver.get(url)    #get url
    search_bar=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')   #xpath to locating search bar
    clear=search_bar.clear()
    search_bar.send_keys(key)       #sending keys
    search_button=driver.find_element_by_xpath('//*[@id="sbtc"]/button/div/span')    #xpath to search button
    search_button.click()  #click search button
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")   #scrolling down to get more images
    time.sleep(5)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

#sending key fruits to find fruits images
sending('fruits')

time.sleep(3)

image=driver.find_elements_by_xpath('//*[@id="islrg"]/div[1]/div/a[1]/div[1]/img')     #xpath to extracting fruits images 
for i in image[:100]:
    Fruits_images.append(i.get_attribute('src'))    #getting attribute from xpath and appending in list

sending('cars')

time.sleep(3)

image=driver.find_elements_by_xpath('//*[@id="islrg"]/div[1]/div/a[1]/div[1]/img')    #xpath to extract car images
for c in image[:100]:
    Car_images.append(c.get_attribute('src'))     #getting attribute from xpath and appending in list

sending('Machine Learning')

time.sleep(3)

image=driver.find_elements_by_xpath('//*[@id="islrg"]/div[1]/div/a[1]/div[1]/img')    #xpath to extract machine learning images
for m in image[:100]:
    Machine_Learning.append(m.get_attribute('src')) #getting attribute from xpath and appending in list

In [34]:
len(Fruits_images),len(Car_images),len(Machine_Learning)

(100, 100, 100)

In [35]:
driver.close()

### Q4. Write a python program to search for a smartphone(e.g: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: "Brand Name","Smartphone name", "Colour","RAM","Storage(ROM)","Primary Camera","Secondary Camera","Display Size","Display Resolution","Processor Cores","Battery Capacity","Price",Product URL". Incase if any of the details is missing then replace it by "-". Save your results in a dataframe and csv

In [2]:
# opening driver 
driver=webdriver.Chrome(r'C:\Users\home\Downloads\chromedriver_win32\chromedriver.exe')
#get url
url='https://www.flipkart.com/'
driver.get(url)
#close log_in window
log_in_pop_up = driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']")
log_in_pop_up.click()

time.sleep(2)
#locating the search bar
search_bar=driver.find_element_by_class_name("_3704LK")
search_bar.clear()
search_bar.send_keys('smartphone')

time.sleep(2)
#locating the button and clicking it to search for sneakers
button=driver.find_element_by_class_name('L0Z3Pu')
button.click()

In [9]:
#creating empty lists
Product_url=[]
Smartphone_name=[]
RAM=[]
Camera=[]
Display_size=[]
Display_resolution=[]
Processor_cores=[]
Battery_capacity=[]
Price=[]
Weight=[]


url=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')   #xpath to find product url
for i in url:
    Product_url.append(i.get_attribute('href'))  #getting attribute from product url

sname=driver.find_elements_by_xpath('//div[@class="_4rR01T"]')  #xpath to find product title details
for s in sname:
    Smartphone_name.append(s.text)

ram=driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]/li[1]')  #xpath to find ram and rom details
for r in ram:
        RAM.append(r.text)
        
dis_size=driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]/li[2]')  #xpath to find display size details
for d in dis_size:
    Display_size.append(d.text)

camera=driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]/li[3]')  #xpath to find camera details
for j in camera:
    Camera.append(j.text)  
    
battery=driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]/li[4]')  #xpath to find battery details
for b in battery:
    Battery_capacity.append(b.text)
    
pro=driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]/li[5]')  #xpath to find  mobile processor details
for d in pro:
    Processor_cores.append(d.text)
    
dis_res=driver.find_elements_by_xpath('//ul[@class="_1xgFaf"]/li[6]')   #xpath to find display resolution details
for r in dis_res:
    Display_resolution.append(r.text)
    
price=driver.find_elements_by_xpath('//div[@class="_30jeq3 _1_WHN1"]')  #xpath to find price
for a in price:
    Price.append(a.text)

    
df=pd.DataFrame({'Modle_Name':Smartphone_name,'RAM/ROM':RAM,'Cam_Details':Camera,
                 'Display_size':Display_size,'Display_resolution':Display_resolution,
                 'Processor_cores':Processor_cores,'Battery_capacity':Battery_capacity,
                 'Price':Price})
df

,Modle_Name,RAM/ROM,Cam_Details,Display_size,Display_resolution,Processor_cores,Battery_capacity,Price
0,"POCO M3 (Cool Blue, 64 GB)",6 GB RAM | 64 GB ROM | Expandable Upto 512 GB,48MP + 2MP + 2MP | 8MP Front Camera,16.59 cm (6.53 inch) Full HD+ Display,"One Year Warranty for Handset, 6 Months for Ac...",Qualcomm Snapdragon 662 Processor,6000 mAh Lithium-ion Polymer Battery,"₹10,499"
1,"REDMI 9i (Midnight Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP Rear Camera | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,Brand Warranty of 1 Year Available for Mobile ...,MediaTek Helio G25 Processor,5000 mAh Lithium Polymer Battery,"₹7,999"
2,"Infinix Hot 10 Play (7° Purple, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,13MP + Depth Sensor | 8MP Front Camera,17.32 cm (6.82 inch) HD+ Display,1 Year on Handset and 6 Months on Accessories,MediaTek Helio G35 Processor,6000 mAh Li-ion Polymer Battery,"₹8,499"
3,"REDMI Note 9 (Pebble Grey, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,48MP + 8MP + 2MP + 2MP | 13MP Front Camera,16.59 cm (6.53 inch) Full HD+ Display,1 Year Manufacturer Warranty for Device and 6 ...,MediaTek Helio G85 Processor,5020 mAh Battery,"₹12,999"
4,"realme C20 (Cool Grey, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,1 Year Warranty for Mobile and 6 Months for Ac...,MediaTek Helio G35 Processor,5000 mAh Battery,"₹6,799"
5,"realme C20 (Cool Blue, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,1 Year Warranty for Mobile and 6 Months for Ac...,MediaTek Helio G35 Processor,5000 mAh Battery,"₹6,799"
6,"REDMI 9i (Sea Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP Rear Camera | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,Brand Warranty of 1 Year Available for Mobile ...,MediaTek Helio G25 Processor,5000 mAh Lithium Polymer Battery,"₹7,999"
7,"realme Narzo 30A (Laser Black, 32 GB)",3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,13MP + 2MP | 8MP Front Camera,16.54 cm (6.51 inch) HD+ Display,1 Year Warranty for Mobile and 6 Months for Ac...,MediaTek Helio G85 Processor,6000 mAh Battery,"₹7,999"
8,"realme Narzo 30A (Laser Blue, 32 GB)",3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,13MP + 2MP | 8MP Front Camera,16.54 cm (6.51 inch) HD+ Display,1 Year Warranty for Mobile and 6 Months for Ac...,MediaTek Helio G85 Processor,6000 mAh Battery,"₹7,999"
9,"realme Narzo 30A (Laser Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,13MP + 2MP | 8MP Front Camera,16.54 cm (6.51 inch) HD+ Display,1 Year Warranty for Mobile and 6 Months for Ac...,MediaTek Helio G85 Processor,6000 mAh Battery,"₹8,999"


In [10]:
driver.close()

### Q5.write a program to scrap geospatial coordinates(latitude,longitude) of a city searched on google maps

In [12]:
#open driver
driver=webdriver.Chrome(r'C:\Users\home\Downloads\chromedriver_win32\chromedriver.exe')

#get url
driver.get('https://www.google.co.in/maps/')

#xpath to find search bar
search_bar=driver.find_element_by_xpath('//input[@class="tactile-searchbox-input"]')
# clearing search_bar
search_bar.clear()
#sending keys to search bar
search_bar.send_keys('Hyderabad')

#xpath to find search button
search_button=driver.find_element_by_xpath('//button[@class="searchbox-searchbutton"]')
#clicking search button
search_button.click()


In [13]:
import re

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)   #printing current url from driver

    lat_lng = re.findall(r'@(.*)data',url_string) #using regrular expression finding url

    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",") #spliting url by ','

        if len(lat_lng_list)>=2:

            lat = lat_lng_list[0] # extracting latitude value

            lng = lat_lng_list[1] #extracting longitude value

        print("Latitude = {}, Longitude = {}".format(lat, lng))   #printing longitude and latitude values



except Exception as e:

        print("Error: ", str(e)) # printing 'e' if there is any error

URL Extracted:  https://www.google.co.in/maps/place/Hyderabad,+Telangana/@17.4121531,78.1278402,10z/data=!3m1!4b1!4m5!3m4!1s0x3bcb99daeaebd2c7:0xae93b78392bafbc2!8m2!3d17.385044!4d78.486671
Latitude = 17.4121531, Longitude = 78.1278402


In [14]:
 driver.close()

### Q6.Write a program to scrap details of all the funding deals for second quarter(i.e July 20-Spetember 20) from trak.in

In [2]:
# opening driver 
driver=webdriver.Chrome(r'C:\Users\home\Downloads\chromedriver_win32\chromedriver.exe')


In [11]:
#url
url='https://trak.in/india-startup-funding-investment-2015/'
driver.get(url) # get url

#creating empty list
date=[]
startup=[]
i_v=[]
sub_ver=[]
city=[]
i_name=[]
invest_type=[]
amount=[]


#loop to scrap the second quater funding details 

for i in range(48,51):
    
    filter=driver.find_element_by_xpath('//*[@id="tablepress-{}_length"]/label/select/option[2]'.format(i)) # xpath to entry filter
    filter.click()

    c1=driver.find_elements_by_xpath('//*[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))  #xpath to scrap date
    for a in c1: 
        date.append(a.text)  
    
    c2=driver.find_elements_by_xpath('//*[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))  #xpath to scrap startup  
    for b in c2:
        startup.append(b.text)
    
    c3=driver.find_elements_by_xpath('//*[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))  #xpath to scrap industry/vertical
    for c in c3:
        i_v.append(c.text)
    
    c4=driver.find_elements_by_xpath('//*[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))  #xpath to scrap sub-vertical
    for d in c4:
        sub_ver.append(d.text)
    
    c5=driver.find_elements_by_xpath('//*[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))  #xpath to scrap city/location
    for e in c5:
        city.append(e.text)
    
    c6=driver.find_elements_by_xpath('//*[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))  #xpath to scrap Investors'name
    for f in c6:
        i_name.append(f.text)
    
    c7=driver.find_elements_by_xpath('//*[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))  #xpath to scrap investment type
    for j in c7:
        invest_type.append(j.text)
    
    c8=driver.find_elements_by_xpath('//*[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))  #xpath to scrap amount 
    for h in c8:
        amount.append(h.text)

        
        
        
#storing extracted data in dataframe 

df=pd.DataFrame({"Date":date,"Startup_Name":startup,"Industry/Vertical":i_v,
                "Sub-Vertical":sub_ver,"City/Location":city,
                "Investors'Name":i_name,"Investment_Type":invest_type,
                 "Amount(In USD)":amount})
df

,Date,Startup_Name,Industry/Vertical,Sub-Vertical,City/Location,Investors'Name,Investment_Type,Amount(In USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [12]:
driver.close()

### Q7.Write a program to scrap all the available detils of top 10 gaming laptops from digit.in.

In [2]:
# open driver
driver=webdriver.Chrome(r'C:\Users\home\Downloads\chromedriver_win32\chromedriver.exe')

In [5]:
#url
url='https://www.digit.in/top-products/best-gaming-laptops-40.html'
#get url
driver.get(url)


# creating emptylist
Title=[]
OS=[]
Display=[]
Processor=[]
Memory=[]
Weight=[]
Dimension=[]
Graphics_pro=[]
Short_dis=[]

tag=driver.find_elements_by_xpath('//h3')  #xpath to find laptop model title
for t in tag:
    Title.append(t.text) # extracted title and appending in list

os=driver.find_elements_by_xpath('/html/body/div[3]/div[1]/div[4]/div/div/table/tbody/tr[3]/td[3]') #xpath to find os
for o in os:
    OS.append(o.text)  #extracting laptop operating system and appending in list 
    
dis=driver.find_elements_by_xpath('/html/body/div[3]/div[1]/div[4]/div/div/table/tbody/tr[4]/td[3]') # xpath to find display details
for d in dis:
    Display.append(d.text)  # extracting and appending the laptop display details in list
    
pro=driver.find_elements_by_xpath('/html/body/div[3]/div[1]/div[4]/div/div/table/tbody/tr[5]/td[3]') #xpath to find laptop processor
for p in pro:
    Processor.append(p.text)  # extracting and appending text of processor details in list
    
memory=driver.find_elements_by_xpath('/html/body/div[3]/div[1]/div[4]/div/div/table/tbody/tr[6]/td[3]')  # xpath to find laptop memory
for m in memory:
    Memory.append(m.text) #extracting and appending text of laptop memory capacity  in list

weight=driver.find_elements_by_xpath('/html/body/div[3]/div[1]/div[4]/div/div/table/tbody/tr[7]/td[3]')  #xpath to find laptop weight
for w in weight:
    Weight.append(w.text) #extracting and appending laptop weight in list
    
dimension=driver.find_elements_by_xpath('/html/body/div[3]/div[1]/div[4]/div/div/table/tbody/tr[8]/td[3]')   # xpath to find dimension of laptop
for d in dimension:
    Dimension.append(d.text) # extracting and appending laptop dimension in list

graph=driver.find_elements_by_xpath('/html/body/div[3]/div[1]/div[4]/div/div/table/tbody/tr[9]/td[3]')   # xpath to find graphics processor
for g in graph:
    Graphics_pro.append(g.text)  # extracting and appending graphics processor in list

s_dis=driver.find_elements_by_xpath('//div[@class="Section-center"]/p') # xpath to find short discription of laptop
for s in s_dis:
    Short_dis.append(s.text) # extracting and appending short discription in list
    
    
    
df=pd.DataFrame({"Title":Title,"OS":OS,"Dislay":Display,"Processor":Processor,"Memory":Memory,"Weight":Weight,
                 "Dimension":Dimension,"Graphics_processor":Graphics_pro,"Short_discription":Short_dis})

df

,Title,OS,Dislay,Processor,Memory,Weight,Dimension,Graphics_processor,Short_discription
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,If you want the absolute best when it comes to...
1,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,If the Area 51M is a bit too much for you then...
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,When the ASUS ROG Strix Scar 15 ended up on ou...
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,The Asus Zephyrus G14 is a first-of-its-kind g...
4,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,Lenovo's Legion 5i is also a suitable laptop f...
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,The machine is powered by an Intel Core i7-108...
6,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,The Acer Aspire 7 gaming laptop is a very capa...


In [6]:
driver.close()